# InfoGAN：基于信息最大化生成对抗网络的可解释特征学习  
  
### GAN的缺陷  

在GAN的生成器中，输入是连续输的噪声向量z，同时不限制生成器使用该噪声的方式。结果，生成器可能以高度纠缠的方式使用噪声，导致z的各个维度与数据的语义特征不对应。  
  
然而很多领域中，是要把变量从语义上划分为各个有意义元素子集的。  
  
如：学习到有用的分离特征可以为以下每个属性分配一组单独的维度：面部表情、眼睛颜色、发型、是否有眼镜以及对应的人的身份。这能有效的帮助那些需要了解数据显著属性的自然任务。  
  
比如：在生成手写数字的时候。一个想法是：模型能够自动的分配一个离散的随机变量去代表生成0-9中哪个数字c1，再选择两个额外的连续变量去代表数字的角度，和数字笔画的胖瘦c2,c3,，然后把剩下的不可压缩的生成变量作为z。  

在InfoGAN中，我们不使用单一的非结构化噪声向量，而是将输入噪声向量分解为两部分：（i）z，它被视为不可压缩噪声的源；（ii）c，我们将其称为潜在代码，并针对数据分布的显著结构化语义特征
  
    
在数学上，我们把所有**分离的变量结构集合**统一标注为：$c_1,c_2,……，c_L$，那么分布$P(c_1,c_2,……，c_L)=\prod_{i=1}^L p(c_i)$    
  
    
<br>  


### InfoGAN中的信息知识  
  
  在InfoGAN中，我们让生成器的输入为各个语义特征的集合。(z，c)，则生成器为：$G(z,c)$。   
  
  即将输入噪声向量分解为两部分：（i）z，它被视为不可压缩噪声的源；（ii）c，我们将其称为潜在代码，并针对数据分布的显著结构化语义特征。  
    
**互信息I(X;Y)=H(X)-H(X|Y)：一个随机变量由于另一个随机变量的原因而导致自身不确定性的减少量。（或者说X由于Y而减少的信息量），**    
    
 **从信息论的角度来看：如过G(z,c)能生成真实样本。那么G(z,c)与C互信息越大。即I(c;G(z,c))越大**  
   
从公式上解释：  
 
  $I(c;G(z,c))=H(c)-H(G(z,c)|c) \qquad \geq 0$   
    
让c已知，那么H(c)已知，则I(c;G(z,c))越大。则H(G(z,c)|c)越小，不确定性越小，表明 G(z,c)由于c的原因，导致自身不确定性在减少。（所以当生成真实样本时，c可以表示我们给定的特殊语义。）  
  
 <br>  
 
 $\color{red}{也就是生成器G要让互信息I(c;G(z,c))最大化，判别器D让互信息I(c;G(z,c))最小化。适当变形后，我们发现这个互信息也可以加入GAN的minmax游戏}$  

<br>

 $$\underset{G}{min }\;\;\underset{D}{max }\;\; V_I(D,G)=V(D,G)-\lambda I(c,G(z,c))$$ 


<br>
     
###   变分法求互信息最大化    

在实际应用中，互信息项I(c;G(z，c))很难直接最大化，因为它需要访问后面的P（c | x）。所以利用变分法来求它的一个最大下界。

 $I(c,G(z,c))=H(c)-H(c|G(z,c))$  
    
 $\qquad\qquad=\sum_{i=1}^n p(c|G(z,c))log\;p(c|G(z,c)) +H(c) $    
   
 $\qquad\qquad=E_{p(c|G(z,c)}log \;p(c|G(z,c)) +H(c) $   
   
 $\qquad\qquad=E_{c\text{~}p（c）}\;[E_{x\text{~}G(z,c)}[E_{c'\text{~}p(c|x)}log \;p(c'|x)]] +H(c) $    
   
设计一个辅助分布Q(c|x)让他接近P(c|x)   
  
 $\qquad\qquad=E_{c\text{~}p（c）}\;[E_{x\text{~}G(z,c)}[E_{c'\text{~}p(c|x)}[log \;p(c'|x)- log \; q(c|x) +log \; q(c|x)]] +H(c) $   
   
$\qquad\qquad=E_{c\text{~}p（c）}\;[E_{x\text{~}G(z,c)}[\underbrace{KL(p(c|x)||q(c|x))}_{\geq 0}+E_{c'\text{~}p(c|x)}[log \; q(c|x)]] +H(c) $   
  
$\qquad\qquad \geq E_{c\text{~}p（c）}\;[E_{x\text{~}G(z,c)}[E_{c'\text{~}p(c'|x)}[log \; q(c'|x)]] +H(c) $   
  
此时找到了互信息的下界。  
   
   
<br>  

<br>  


**引理：**  
对于随机变量X，Y和函数f(x,y)，在适当的条件下：$E_{x\text{~}X,y\text{~}Y|x}[f(x,y)]=E_{x\text{~}X，y\text{~}Y|x，x'\text{~}X|y}\;\;[f(x',y)]$  
  
<br>   


则对上面的下界使用引理：  
$I(c,G(z,c)) \geq  E_{c\text{~}p（c）}\;[E_{x\text{~}G(z,c)}[E_{c'\text{~}p(c'|x)}[log \; q(c'|x)]] +H(c) $   
    
  
$\qquad\qquad = E_{c\text{~}p(c)，x\text{~}G(z,c)}\;[log \; q(c|x)]+H(c) $   
  
$\qquad\qquad=L_I(G,Q)$    
  
**$L_I(G,Q)$的显示表达式：可以MCMC采样模拟;(使用引理后，我们消除了重后验概率抽样的必要)**    
  
<br>    
  
<br> 

### 最总的目标函数和InfoGAN 的结构  

**最总的目标函数为：**  
  
  
$\underset{G,Q}{min}\;\underset{D}{max}\; V_{InfoGAN}(D,G,Q) \;=\; V(D,G)-\lambda L_I(G,Q)$      


 在实际中，通常把辅助分布Q设为一个神经网络。在InfoGAN中，我们让Q和D共享卷积层，但是有各自不同的输出层。D的顶层输出真假。Q的顶层为全连接层。  
   这样做的好处是：InfoGAN只给GAN增加了微不足道的计算成本
   
   
 ![](img/1.webp)  
  